In [ ]:
## Want to try a bulk download example?

## This is just to get random words generated. 
from nltk.corpus import words
import random



import requests
from tqdm import tqdm
import datetime

import os
from multiprocessing import Lock
from concurrent.futures import ThreadPoolExecutor
from timeit import default_timer as timer
from copy import deepcopy
from time import sleep

payloadtemplate = {
    "maintext1": "Placeholder",
    "shape1": "facebook",
    "textcolor1": "black",
    "transparency1": "medium"    
}


In [ ]:
baseurl = "http://localhost:5000/generate/"

downloaddir = "downloads/"
os.makedirs(downloaddir, exist_ok=True)

# textlog = {}

# masterlock = Lock()

In [ ]:
# It takes a while to generate the sample texts. This is where you want to bring in your texts.
print(f"Generating sample texts. This will take a while.")
targetentries = []
for i in tqdm(range(0, 1600)):
    wordswanted = random.randrange(3, 30)
    rand_words = ' '.join(random.sample(words.words(), wordswanted))
    filename = downloaddir + rand_words.split()[0] + str(len(targetentries)) + ".jpg"
    entry = (rand_words, filename)
    targetentries.append(entry)

In [ ]:
def send_text(mytext, myfilename):
    global textlog
    global masterlock
    mypayload = deepcopy(payloadtemplate)
    mypayload['maintext1'] = mytext
    sleep(random.random()/50)   # Sleep a tiny bit of a second, to try to stagger our requests just a hair
    r = requests.get(baseurl, data=mypayload, timeout=15)
    while r.status_code != 200:
        sleep(random.random()/10)  # Sleep a fraction of a second
        r = requests.get(baseurl, data=mypayload, timeout=15)
    # filename = downloaddir + r.headers['Content-Disposition'].split("filename=")[-1]
    filename = myfilename
    if os.path.exists(filename):      # Do not clobber an existing file, for all sorts of reasons
        print(f"Will NOT overwrite {filename}")
    else:
        with open(filename, "wb") as outfile:
            outfile.write(r.content)
        # masterlock.acquire()
        # textlog[filename] = payload
        # masterlock.release()
    return()

In [ ]:
print(f"Trying to publish {len(targetentries):,}")

starttime = timer()

with ThreadPoolExecutor(max_workers=12) as executor:
    for targetentry in targetentries:
        targettext, targetfilename = targetentry
        result = executor.submit(send_text, targettext, targetfilename)
        
endtime = timer()
print(f"Time to publish {len(targetentries):,}: {(endtime-starttime):,} seconds")